In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [69]:
# Read cleaned data
packs_team = pd.read_csv(r'puzzle-scheduling/EDA/packs_df.csv', index_col=0)
packs_mine = pd.read_csv(r'data/packs_clean.csv', index_col=0)
members = pd.read_csv(r'puzzle-scheduling/EDA/member_df.csv', index_col=0)

In [70]:
packs_team.head()

pack_name           brand  \
0  Anne Belle Thompson The Mikado Anne Belle Thom...  Other-Hand-cut   
1  Anthology Puzzles Alphonse Mucha La Plume Anth...             NaN   
2  Anthology Puzzles Colorful Cat Anthology Puzzl...             NaN   
3  Anthology Puzzles Framed American Gothic Antho...             NaN   
4  Anthology Puzzles Over The Moon Anthology Puzz...             NaN   

   piece_count difficulty_rating  
0        387.0           Average  
1        320.0           Average  
2        150.0           Average  
3        320.0           Average  
4        278.0              Hard

In [71]:
packs_mine.head()

pack_name         brand_0 brand_1  \
0  Anne Belle Thompson The Mikado Anne Belle Thom...  Other-Hand-cut     NaN   
1  Anthology Puzzles Alphonse Mucha La Plume Anth...             NaN     NaN   
2  Anthology Puzzles Colorful Cat Anthology Puzzl...             NaN     NaN   
3  Anthology Puzzles Framed American Gothic Antho...             NaN     NaN   
4  Anthology Puzzles Over The Moon Anthology Puzz...             NaN     NaN   

   pieces_0  pieces_1   diff_0   diff_1  num_puzzles  
0     387.0     242.0  Average  Average            2  
1     320.0     160.0  Average  Average            2  
2     150.0     170.0  Average  Average            2  
3     320.0     300.0  Average  Average            2  
4     278.0     177.0     Hard   A-Easy            2

In [72]:
members.head()

memberID   holdtime                                         puzzlepack  \
0  member1   2.939411  Artifact Puzzles Justin Hillgrove Word Travels...   
1  member1   0.998885  DaVici Puzzles Full Moon Feast DaVici Puzzles ...   
2  member1  10.865032  DaVici Puzzles Flying Frigate DaVici Puzzles H...   
3  member1  22.083971  Liberty Puzzles Haeckel Hummingbirds Nautilus ...   
4  member1   5.077603          DaVici Puzzles Diana Zimens City Of Cats    

   num_puzzles     brand  
0            2  Artifact  
1            2    DaVici  
2            2    DaVici  
3            2   Liberty  
4            1    DaVici

## Pack Features

Can packs be split?

    1. If packs cannot be split then no point considering the two puzzles as separate, create synthetic features for the pack
    2. Alternatively could break things down to 'puzzle' level, but increases search space possibly needlessly if packs cannot be split
    
    
Per pack
    1. Cummulative pieces (maybe map pieces to difficulty, or have synth weight)
    2. Cummulative difficulty 
    3. Number of puzzles in pack
    4. Brand?

In [73]:
packs_mine[packs_mine['diff_0'] == 'nan']

Empty DataFrame
Columns: [pack_name, brand_0, brand_1, pieces_0, pieces_1, diff_0, diff_1, num_puzzles]
Index: []

In [74]:
# Create synthetic pack level features combining information about the two possible puzzles in the pack

difficulty_mapping = {'A-Easy': 1, 'Average': 2, 'Hard': 3, 'Really-Hard': 4}

packs_mine['diff_0'] = packs_mine['diff_0'].map(lambda x: difficulty_mapping[x], na_action='ignore')
packs_mine['diff_1'] = packs_mine['diff_1'].map(lambda x: difficulty_mapping[x], na_action='ignore')

In [75]:
packs_mine.head()

pack_name         brand_0 brand_1  \
0  Anne Belle Thompson The Mikado Anne Belle Thom...  Other-Hand-cut     NaN   
1  Anthology Puzzles Alphonse Mucha La Plume Anth...             NaN     NaN   
2  Anthology Puzzles Colorful Cat Anthology Puzzl...             NaN     NaN   
3  Anthology Puzzles Framed American Gothic Antho...             NaN     NaN   
4  Anthology Puzzles Over The Moon Anthology Puzz...             NaN     NaN   

   pieces_0  pieces_1  diff_0  diff_1  num_puzzles  
0     387.0     242.0     2.0     2.0            2  
1     320.0     160.0     2.0     2.0            2  
2     150.0     170.0     2.0     2.0            2  
3     320.0     300.0     2.0     2.0            2  
4     278.0     177.0     3.0     1.0            2

## These packs have more information than there are puzzles, this is weird

In [76]:
# No piece information
packs_mine[(packs_mine.pieces_1.notna() | packs_mine.diff_1.notna()) & (packs_mine.num_puzzles == 1)]

pack_name   brand_0 brand_1  \
10           Artifact Puzzles Bek Cruddace Europe Map   Artifact     NaN   
45   Artifact Puzzles Roch Urbaniak Floating Town A...  Artifact     NaN   
46    Artifact Puzzles Sandi Rigby Botanical Dreaming   Artifact     NaN   
77            DaVici Puzzles Awakening DaVici Puzzles     DaVici     NaN   
90              DaVici Puzzles Under the Red Umbrella     DaVici     NaN   
..                                                 ...       ...     ...   
846  Artifact Puzzles Seurat Grande Jatte Artifact ...  Artifact     NaN   
864          DaVici Puzzles Diana Zimens City Of Cats     DaVici     NaN   
868          Artifact Puzzles Jethro Buck Wild Things   Artifact     NaN   
874  Ecru Puzzles William Penhallow Henderson Lucer...  Artifact    Ecru   
897            Artifact Puzzles Great Robot Migration   Artifact     NaN   

     pieces_0  pieces_1  diff_0  diff_1  num_puzzles  
10      900.0       NaN     2.0     2.0            1  
45      228.0     216.0     1.0     1.0            1  
46      273.0       NaN     3.0     2.0            1  
77      450.0     280.0     2.0     2.0            1  
90      600.0       NaN     3.0     2.0            1  
..        ...       ...     ...     ...          ...  
846     428.0     338.0     4.0     3.0            1  
864     700.0       NaN     2.0     2.0            1  
868     539.0       NaN     3.0     2.0            1  
874     385.0     411.0     1.0     1.0            1  
897     480.0       NaN     3.0     2.0            1  

[167 rows x 8 columns]

In [77]:
packs_raw = pd.read_csv('data/packs.tsv', sep='\t', header=None)
packs_raw.head()

0               1        2  \
0  Anne Belle Thompson The Mikado Anne Belle Thom...  Other-Hand-cut  387,242   
1  Anthology Puzzles Alphonse Mucha La Plume Anth...             NaN  320,160   
2  Anthology Puzzles Colorful Cat Anthology Puzzl...             NaN  150,170   
3  Anthology Puzzles Framed American Gothic Antho...             NaN  320,300   
4  Anthology Puzzles Over The Moon Anthology Puzz...             NaN  278,177   

                 3  
0  Average,Average  
1  Average,Average  
2  Average,Average  
3  Average,Average  
4      Hard,A-Easy

In [78]:
weirdos = packs_raw[(packs_mine.pieces_0.notna()) & (packs_mine.pieces_1.isna()) & (packs_mine.num_puzzles == 2)]
weirdos.head()
weirdos.to_csv(r'data/two_puzzles_no_second_piece_count.csv', index=False)

In [79]:
# Packs that are missing only one of piece/difficulty for the second puzzle
packs_mine[((packs_mine.pieces_1.notna() & packs_mine.diff_1.isna()) | (packs_mine.pieces_1.isna() & packs_mine.diff_1.notna())) & (packs_mine.num_puzzles == 2)]

pack_name    brand_0  \
9    Artifact Puzzles Bee Eater Artifact Puzzles Ke...   Artifact   
33   Artifact Puzzles Kristian Adams Snoozies Artif...   Artifact   
133  Liberty Puzzle Konstatin Gorbatov A Winters Da...       Ecru   
142  Liberty Puzzles Alphonse Mucha La Danse Libert...    Liberty   
243  Liberty Puzzles Muktair Oladoja Lovers Ecru Pu...       Ecru   
248  Liberty Puzzles Paris Air France Ecru Puzzles ...       Ecru   
389  Snowflake Puzzles Colorful Twirly-ques Ecru Pu...       Ecru   
427  Stave Puzzles My Special Tea Wentworth Puzzles...      Stave   
440  Turtle Teasers The Paradise of the Medicine Bu...        NaN   
456  Wentworth Puzzles Beautiful Victorian Day Whim...  Wentworth   
652  Artifact Puzzles Paul Ranson Flowers Double Si...   Artifact   
672  Artifact Puzzles Allie Sullberg Lets Dance Art...   Artifact   
673  Artifact Puzzles Alceste Campriani Pappagalli ...   Artifact   
686  Ecru Puzzles Allen Gilbert Cram Fishermans Cot...   Artifact   
690  Ecru Puzzles Kawase Hasui Fall Ecru Puzzles Ho...       Ecru   

             brand_1  pieces_0  pieces_1  diff_0  diff_1  num_puzzles  
9                NaN     439.0       NaN     3.0     1.0            2  
33               NaN     413.0       NaN     2.0     2.0            2  
133          Liberty     527.0       NaN     3.0     2.0            2  
142              NaN     258.0       NaN     2.0     3.0            2  
243          Liberty     504.0       NaN     2.0     2.0            2  
248          Liberty     483.0       NaN     3.0     1.0            2  
389   Other-Hand-cut     500.0       NaN     2.0     1.0            2  
427        Wentworth      75.0       NaN     4.0     1.0            2  
440              NaN     158.0       NaN     2.0     2.0            2  
456  Other-Laser-cut     450.0       NaN     2.0     2.0            2  
652              NaN      79.0       NaN     3.0     3.0            2  
672              NaN     367.0       NaN     1.0     2.0            2  
673              NaN     201.0       NaN     2.0     2.0            2  
686             Ecru     158.0       NaN     2.0     2.0            2  
690              NaN       NaN       NaN     1.0     2.0            2

In [80]:
# Packs that claim 2 puzzles but have no info on second, but do have info on the first
packs_mine[(packs_mine.pieces_1.isna() & packs_mine.diff_1.isna()) & (packs_mine.num_puzzles == 2) & (packs_mine.pieces_0.notna() | packs_mine.diff_0.notna())]

Empty DataFrame
Columns: [pack_name, brand_0, brand_1, pieces_0, pieces_1, diff_0, diff_1, num_puzzles]
Index: []

In [81]:
first_puzzle = packs_mine[['pieces_0', 'diff_0']]
second_puzzle = packs_mine[['pieces_1', 'diff_1']].rename({'pieces_1': 'pieces_0', 'diff_1': 'diff_0'}, axis=1)
puzzles_stacked = pd.concat([first_puzzle, second_puzzle], ignore_index=True).dropna()
puzzles_stacked.head()

pieces_0  diff_0
0     387.0     2.0
1     320.0     2.0
2     150.0     2.0
3     320.0     2.0
4     278.0     3.0

In [82]:
single_puzzle_packs = packs_mine[packs_mine.num_puzzles == 1].dropna(subset=['pieces_0', 'diff_0'])
two_puzzle_packs = packs_mine[packs_mine.num_puzzles == 2].dropna(subset=['pieces_0', 'diff_0', 'pieces_1', 'diff_1'])

In [83]:
valid_packs = pd.concat([single_puzzle_packs, two_puzzle_packs])
valid_packs.head()

pack_name   brand_0 brand_1  \
10          Artifact Puzzles Bek Cruddace Europe Map   Artifact     NaN   
45  Artifact Puzzles Roch Urbaniak Floating Town A...  Artifact     NaN   
46   Artifact Puzzles Sandi Rigby Botanical Dreaming   Artifact     NaN   
77           DaVici Puzzles Awakening DaVici Puzzles     DaVici     NaN   
90             DaVici Puzzles Under the Red Umbrella     DaVici     NaN   

    pieces_0  pieces_1  diff_0  diff_1  num_puzzles  
10     900.0       NaN     2.0     2.0            1  
45     228.0     216.0     1.0     1.0            1  
46     273.0       NaN     3.0     2.0            1  
77     450.0     280.0     2.0     2.0            1  
90     600.0       NaN     3.0     2.0            1

In [84]:
pieces_by_difficulty = []

for i, row in valid_packs.iterrows():
    out = {
        'name': row['pack_name'],
        'pieces_d1': 0,
        'pieces_d2': 0,
        'pieces_d3': 0,
        'pieces_d4': 0,
        'num_puzzles': row['num_puzzles']
    }
    
    out[f'pieces_d{str(int(row["diff_0"]))}'] = row['pieces_0']
    
    if(row['num_puzzles'] == 2):
        out[f'pieces_d{str(int(row["diff_1"]))}'] += row['pieces_1']
        
    pieces_by_difficulty.append(out)


In [85]:
pbyd = pd.DataFrame(pieces_by_difficulty)
pbyd.head()

name  pieces_d1  pieces_d2  \
0          Artifact Puzzles Bek Cruddace Europe Map         0.0      900.0   
1  Artifact Puzzles Roch Urbaniak Floating Town A...      228.0        0.0   
2   Artifact Puzzles Sandi Rigby Botanical Dreaming         0.0        0.0   
3           DaVici Puzzles Awakening DaVici Puzzles         0.0      450.0   
4             DaVici Puzzles Under the Red Umbrella         0.0        0.0   

   pieces_d3  pieces_d4  num_puzzles  
0        0.0        0.0            1  
1        0.0        0.0            1  
2      273.0        0.0            1  
3        0.0        0.0            1  
4      600.0        0.0            1

In [86]:
valid_packs[valid_packs.brand_1.notna()]

pack_name         brand_0  \
99   Davici Puzzles Photo Album Ecru Puzzles Sunny ...          DaVici   
203  Liberty Puzzles Jane Tattersfield Peacock And ...         Liberty   
606  Nautilus Puzzles Liberty Eugene Delacroix Libe...         Liberty   
651  Artifact Puzzles Iwona Lifsches Santa Claus De...        Artifact   
874  Ecru Puzzles William Penhallow Henderson Lucer...        Artifact   
..                                                 ...             ...   
863  Ecru Puzzles William Penhallow Henderson Lucer...        Artifact   
865  Nikolyaaa Red Boats Wheelgiant Balancing Bould...  Other-Hand-cut   
871  Ecru Puzzles Rachell Sumpter Wild Moor Artifac...        Artifact   
884  Ecru Puzzles Jonik Sunny November Artifact Puz...        Artifact   
907  Ecru Puzzles Allen Gilbert Cram Fishermans Cot...        Artifact   

             brand_1  pieces_0  pieces_1  diff_0  diff_1  num_puzzles  
99              Ecru     600.0     369.0     2.0     2.0            1  
203  Other-Laser-cut     508.0     400.0     2.0     2.0            1  
606         Nautilus     466.0       NaN     3.0     2.0            1  
651        Wentworth     158.0     250.0     1.0     3.0            1  
874             Ecru     385.0     411.0     1.0     1.0            1  
..               ...       ...       ...     ...     ...          ...  
863             Ecru     385.0     482.0     3.0     2.0            2  
865  Other-Laser-cut     249.0     425.0     2.0     2.0            2  
871             Ecru     260.0     338.0     3.0     2.0            2  
884             Ecru     369.0     311.0     2.0     2.0            2  
907             Ecru     158.0     205.0     2.0     3.0            2  

[105 rows x 8 columns]

In [87]:
packs_dropped = pd.read_csv(r'data/packs_cleaned_dropna.csv', index_col=0)
packs_dropped.head()

pack_name         brand_0 brand_1  \
0  Anne Belle Thompson The Mikado Anne Belle Thom...  Other-Hand-cut     NaN   
1  Anthology Puzzles Alphonse Mucha La Plume Anth...             NaN     NaN   
2  Anthology Puzzles Colorful Cat Anthology Puzzl...             NaN     NaN   
3  Anthology Puzzles Framed American Gothic Antho...             NaN     NaN   
4  Anthology Puzzles Over The Moon Anthology Puzz...             NaN     NaN   

   pieces_0  pieces_1   diff_0   diff_1  num_puzzles  
0     387.0     242.0  Average  Average            2  
1     320.0     160.0  Average  Average            2  
2     150.0     170.0  Average  Average            2  
3     320.0     300.0  Average  Average            2  
4     278.0     177.0     Hard   A-Easy            2

In [88]:
# Create synthetic pack level features combining information about the two possible puzzles in the pack

difficulty_mapping = {'A-Easy': 1, 'Average': 2, 'Hard': 3, 'Really-Hard': 4}

packs_dropped['diff_0'] = packs_dropped['diff_0'].map(lambda x: difficulty_mapping[x], na_action='ignore')
packs_dropped['diff_1'] = packs_dropped['diff_1'].map(lambda x: difficulty_mapping[x], na_action='ignore')

In [89]:
pieces_by_difficulty = []

for i, row in packs_dropped.iterrows():
    out = {
        'name': row['pack_name'],
        'pieces_d1': 0,
        'pieces_d2': 0,
        'pieces_d3': 0,
        'pieces_d4': 0,
        'num_puzzles': row['num_puzzles']
    }
    
    if(row['pieces_0'].notna()]:
        out[f'pieces_d{str(int(row["diff_0"]))}'] += row['pieces_0']
    
    if(row['num_puzzles'] == 2 and row['pieces_1'].notna()):
        out[f'pieces_d{str(int(row["diff_1"]))}'] += row['pieces_1']
        
    pieces_by_difficulty.append(out)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (2778832469.py, line 13)

In [90]:
pieces_df = pd.DataFrame(pieces_by_difficulty)
pieces_df.head()

name  pieces_d1  pieces_d2  \
0          Artifact Puzzles Bek Cruddace Europe Map         0.0      900.0   
1  Artifact Puzzles Roch Urbaniak Floating Town A...      228.0        0.0   
2   Artifact Puzzles Sandi Rigby Botanical Dreaming         0.0        0.0   
3           DaVici Puzzles Awakening DaVici Puzzles         0.0      450.0   
4             DaVici Puzzles Under the Red Umbrella         0.0        0.0   

   pieces_d3  pieces_d4  num_puzzles  
0        0.0        0.0            1  
1        0.0        0.0            1  
2      273.0        0.0            1  
3        0.0        0.0            1  
4      600.0        0.0            1

In [91]:
pieces_df.to_csv(r'data/pieces_by_puzzle_and_difficulty.csv')

In [92]:
packs_dropped[packs_dropped.pieces_1.isna() | packs_dropped.pieces_0.isna()]

pack_name   brand_0 brand_1  \
10           Artifact Puzzles Bek Cruddace Europe Map   Artifact     NaN   
46    Artifact Puzzles Sandi Rigby Botanical Dreaming   Artifact     NaN   
90              DaVici Puzzles Under the Red Umbrella     DaVici     NaN   
92                                    DaVici Red Book     DaVici     NaN   
95    Davici Puzzles Claude Monet Woman in the Garden     DaVici     NaN   
..                                                 ...       ...     ...   
789                   Artifact Puzzles Bokuyo Forest    Artifact     NaN   
808  Artifact Puzzles Hieronymus Bosch Garden Of Ea...  Artifact     NaN   
864          DaVici Puzzles Diana Zimens City Of Cats     DaVici     NaN   
868          Artifact Puzzles Jethro Buck Wild Things   Artifact     NaN   
897            Artifact Puzzles Great Robot Migration   Artifact     NaN   

     pieces_0  pieces_1  diff_0  diff_1  num_puzzles  
10      900.0       NaN       2       2            1  
46      273.0       NaN       3       2            1  
90      600.0       NaN       3       2            1  
92      700.0       NaN       3       2            1  
95      552.0       NaN       1       2            1  
..        ...       ...     ...     ...          ...  
789     689.0       NaN       4       2            1  
808     529.0       NaN       2       2            1  
864     700.0       NaN       2       2            1  
868     539.0       NaN       3       2            1  
897     480.0       NaN       3       2            1  

[152 rows x 8 columns]

In [93]:
pieces_df[pieces_df.isna().any(axis=1)]

Empty DataFrame
Columns: [name, pieces_d1, pieces_d2, pieces_d3, pieces_d4, num_puzzles]
Index: []